In [1]:
import os
from pathlib import Path
from typing import List, Dict, Any, Tuple

import pandas as pd
import numpy as np
from nltk.corpus import stopwords

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import umap
import hdbscan

In [2]:
##############
# parameters #
##############

DATA_PATH = Path(r"C:\Users\linna\OneDrive\Documents\Python_Dev\topic-modeling\data\public_comments.json")

TEXT_COL = "comment_text"
DOC_ID_COL = "comment_id"
DOCKET_TO_USE = "TTB-2025-0003"

# repo / outputs
try:
    REPO_ROOT = Path(__file__).parent.parent.resolve()
except NameError:
    REPO_ROOT = Path(os.getcwd()).parent.resolve()

OUTPUTS_DIR = REPO_ROOT / "outputs"
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

TOPIC_SUMMARY_CSV = OUTPUTS_DIR / "bertopic_topic_summary.csv"
OUTPUT_DF_CSV = OUTPUTS_DIR / "comments_with_bertopic.csv"
MODEL_SAVE_FILE = Path(r"C:\Users\linna\Documents\bertopic_model")

# for later dev -- want to remove nonsense tokens AFTER topics are computed:
# STOPWORDS = set(stopwords.words('english'))
# BAD_DISPLAY_TOKENS = {"/sbr/s", "brand39s", str(STOPWORDS)}

In [3]:
def load_data(path: Path) -> pd.DataFrame:
    df = pd.read_json(path, orient="records", lines=False)
    if TEXT_COL not in df.columns:
        raise ValueError(f"{TEXT_COL} not found in dataframe columns: {df.columns.tolist()}")
    df = df.dropna(subset=[TEXT_COL]).reset_index(drop=True)
    print(f"Loaded {len(df)} comments from {path}")
    return df

def filter_by_docket(df: pd.DataFrame, docket: str | None) -> pd.DataFrame:
    if docket is None:
        return df
    if "docket_id" not in df.columns:
        raise ValueError("docket_id column not in dataframe")
    df_sub = df[df["docket_id"] == docket].reset_index(drop=True)
    print(f"Filtered to docket '{docket}': {len(df_sub)} comments")
    return df_sub

def chunk_documents(documents: List[str], chunk_size: int = 200, overlap: int = 50) -> Tuple[List[str], List[str]]:
    """
    Simple sliding-window chunker returning (chunks, chunk_origin_doc_id)
    chunk_size = number of tokens (approx via split) per chunk
    overlap = overlapping tokens between chunks
    """
    chunks = []
    origin_ids = []
    for i, doc in enumerate(documents):
        toks = str(doc).split()
        if len(toks) <= chunk_size:
            chunks.append(" ".join(toks))
            origin_ids.append(i)
        else:
            start = 0
            while start < len(toks):
                end = min(len(toks), start + chunk_size)
                chunks.append(" ".join(toks[start:end]))
                origin_ids.append(i)
                if end == len(toks):
                    break
                start = end - overlap
    return chunks, origin_ids

def train_bertopic(documents: List[str], embedding_model_name: str, verbose: bool = True) -> Tuple[BERTopic, np.ndarray]:
    """
    Returns (model, embeddings)
    """
    if verbose:
        print("Loading embedding model:", embedding_model_name)
    embedder = SentenceTransformer(embedding_model_name)

    # compute embeddings first (we can reuse them)
    if verbose:
        print("Computing embeddings for", len(documents), "documents...")
    embeddings = embedder.encode(documents, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

    # build UMAP and HDBSCAN instances with tunable params
    umap_model = umap.UMAP(n_neighbors=UMAP_N_NEIGHBORS, n_components=UMAP_N_COMPONENTS, min_dist=UMAP_MIN_DIST, metric="cosine", random_state=42)
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE, min_samples=HDBSCAN_MIN_SAMPLES, metric="euclidean", cluster_selection_method="eom", prediction_data=True)

    # instantiate BERTopic with our reducers/clusters
    topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=True, verbose=verbose)
    if verbose:
        print("Training BERTopic...")
    topics, probs = topic_model.fit_transform(documents, embeddings)
    if verbose:
        print("BERTopic training complete. Generated", len(set(topics)) - (1 if -1 in topics else 0), "non-outlier topics (excludes -1).")
    return topic_model, embeddings

def map_dominant_and_topN_bertopic(model: BERTopic, documents: List[str], df: pd.DataFrame,
                                   doc_id_col: str, embeddings: np.ndarray | None = None,
                                   topN: int = 3) -> pd.DataFrame:
    """
    Adds:
      - bertopic_dominant_topic
      - bertopic_top_topics (list of best-represented topics in comment)
      - bertopic_topic_rank_{i}
    Pass 'embeddings' (precomputed) to avoid BERTopic trying to re-embed.
    """
    # failsafe if (for some reason) we did not already compute/pass embeddings:
    if embeddings is not None:
        topics, probs = model.transform(documents, embeddings=embeddings)
    else:
        topics, probs = model.transform(documents)

    df["bertopic_dominant_topic"] = topics

    # compute topN from probs if available
    if probs is not None:
        probs_arr = np.array(probs)
        if probs_arr.ndim == 2:
            top_indices = np.argsort(probs_arr, axis=1)[:, ::-1][:, :topN]
            top_lists = top_indices.tolist()
            topic_info = model.get_topic_info().reset_index(drop=True)
            topic_ids_order = topic_info["Topic"].tolist()
            idx_to_topic = {i: tid for i, tid in enumerate(topic_ids_order)}
            top_topics = [[idx_to_topic.get(i, -1) for i in lst] for lst in top_lists]
            df["bertopic_top_topics"] = top_topics
            for i in range(topN):
                df[f"bertopic_topic_rank_{i+1}"] = df["bertopic_top_topics"].apply(lambda l: l[i] if i < len(l) else -1)
        else:
            df["bertopic_top_topics"] = df["bertopic_dominant_topic"].apply(lambda x: [int(x)] + [-1]*(topN-1))
            for i in range(topN):
                df[f"bertopic_topic_rank_{i+1}"] = df["bertopic_top_topics"].apply(lambda l: l[i] if i < len(l) else -1)
    else:
        df["bertopic_top_topics"] = df["bertopic_dominant_topic"].apply(lambda x: [int(x)] + [-1]*(topN-1))
        for i in range(topN):
            df[f"bertopic_topic_rank_{i+1}"] = df["bertopic_top_topics"].apply(lambda l: l[i] if i < len(l) else -1)

    return df

def build_topic_summary_bertopic(model: BERTopic, df: pd.DataFrame, documents: List[str],
                                 doc_id_col: str, top_words: int = 10, sample_docs: int = 5) -> pd.DataFrame:
    """
    Build a topic summary df for output with columns: topic_num, size, top_words, sample_comments
    """
    info = model.get_topic_info()  # df with 'topic', 'count' and 'name'
    rows = []
    for _, row in info.iterrows():
        tnum = int(row["Topic"])
        size = int(row["Count"])

        # get top words for topic (BERTopic returns list of (word, score))
        topic_words = model.get_topic(tnum)
        if topic_words:
            words = [w for w, s in topic_words][:top_words]
        else:
            words = []

        # get representative comments
        rep_docs_list = []
        try:
            rep = model.get_representative_docs(tnum)
            # handle multiple output configurations
            if rep is None:
                rep_docs_list = []
            elif isinstance(rep, (list, tuple)):
                rep_docs_list = list(rep)[:sample_docs]
            else:
                try:
                    rep_docs_list = list(rep)[:sample_docs]
                except Exception:
                    rep_docs_list = []
        except Exception:
            rep_docs_list = []

        # if no representative docs from model grab df rows for that topic instead
        if not rep_docs_list:
            try:
                mask = df["bertopic_dominant_topic"] == tnum
                rep_docs_list = df.loc[mask, TEXT_COL].astype(str).tolist()[:sample_docs]
            except Exception:
                rep_docs_list = []

        # format sample comments
        sample_texts = []
        for s in rep_docs_list:
            try:
                s_str = str(s).replace("\n", " ")
            except Exception:
                s_str = ""
            sample_texts.append(s_str[:400])

        rows.append({
            "topic_num": tnum,
            "size": size,
            "top_words": ", ".join(words),
            "sample_comments": " ||| ".join(sample_texts)
        })

    summary_df = pd.DataFrame(rows).sort_values("size", ascending=False).reset_index(drop=True)
    return summary_df

In [4]:
CHUNK_DOCS = True              
CHUNK_SIZE = 80                # tokens per chunk (words)
CHUNK_OVERLAP = 20             # overlapping tokens between chunks
HDBSCAN_MIN_CLUSTER_SIZE = 3   # 3-5 usually serves well, start tuning here. Also adjust UMAP_MIN_DISTANCE & N_COMPONENTS
HDBSCAN_MIN_SAMPLES = 1
UMAP_N_NEIGHBORS = 10
UMAP_MIN_DIST = 0.05
UMAP_N_COMPONENTS = 5
EMBEDDING_MODEL = "all-mpnet-base-v2"

TOP_WORDS_PER_TOPIC = 7
SAMPLE_DOCS_PER_TOPIC = 5

In [6]:
def chunk_documents(documents: List[str], chunk_size: int = 80, overlap: int = 20):
    chunks = []
    origin_ids = []
    for i, doc in enumerate(documents):
        toks = str(doc).split()
        if len(toks) <= chunk_size:
            chunks.append(" ".join(toks))
            origin_ids.append(i)
        else:
            start = 0
            while start < len(toks):
                end = min(len(toks), start + chunk_size)
                chunks.append(" ".join(toks[start:end]))
                origin_ids.append(i)
                if end == len(toks):
                    break
                start = end - overlap
    return chunks, origin_ids

def train_bertopic_on_chunks(documents: List[str], embedding_model_name: str,
                             chunk_size=80, overlap=20,
                             umap_n_neighbors=10, umap_min_dist=0.0, umap_n_components=5,
                             hdb_min_cluster_size=3, hdb_min_samples=1,
                             verbose=True):
    # chunk
    chunks, origin_ids = chunk_documents(documents, chunk_size=chunk_size, overlap=overlap)
    print(f"Created {len(chunks)} chunks from {len(documents)} documents (ratio {len(chunks)/len(documents):.2f}).")

    # embed
    from sentence_transformers import SentenceTransformer
    embedder = SentenceTransformer(embedding_model_name)
    embeddings = embedder.encode(chunks, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

    # build UMAP/HDBSCAN
    import umap
    import hdbscan
    umap_model = umap.UMAP(n_neighbors=umap_n_neighbors, n_components=umap_n_components,
                           min_dist=umap_min_dist, metric="cosine", random_state=42)
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=hdb_min_cluster_size,
                                    min_samples=hdb_min_samples,
                                    metric="euclidean", cluster_selection_method="eom",
                                    prediction_data=True)
    from bertopic import BERTopic
    topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model,
                           calculate_probabilities=True, verbose=verbose)

    # fit
    topics, probs = topic_model.fit_transform(chunks, embeddings)
    print("BERTopic (chunks) training complete. Non-outlier topics:",
          len(set(topics)) - (1 if -1 in topics else 0))
    return topic_model, chunks, origin_ids, embeddings, topics, probs

def aggregate_chunk_topics_to_docs(chunks, origin_ids, chunk_topics, chunk_probs, df, doc_id_col, topN=3):
    """
    Aggregates chunk-level topics back to doc-level.
    - Majority vote on chunk topics per document for dominant topic.
    - For topN, uses chunk counts per topic (you can extend to use probabilities).
    """
    import pandas as _pd
    tmp = _pd.DataFrame({
        "origin_idx": origin_ids,
        "chunk_topic": chunk_topics
    })
    # count topic frequency per original doc index
    counts = tmp.groupby(["origin_idx", "chunk_topic"]).size().rename("cnt").reset_index()
    # find dominant topic per origin_idx
    dominant = counts.sort_values(["origin_idx", "cnt"], ascending=[True, False]).groupby("origin_idx").first().reset_index()
    dominant_map = dict(zip(dominant["origin_idx"], dominant["chunk_topic"]))

    # build topN lists per origin_idx
    topn_map = {}
    for oid, group in counts.groupby("origin_idx"):
        top_topics = group.sort_values("cnt", ascending=False).head(topN)["chunk_topic"].astype(int).tolist()
        # pad if needed
        while len(top_topics) < topN:
            top_topics.append(-1)
        topn_map[oid] = top_topics

    # attach to df by original index
    # original index in df corresponds to 0..len(df)-1 as created in main flow
    df = df.copy()
    df["bertopic_dominant_topic"] = df.index.map(lambda i: int(dominant_map.get(i, -1)))
    df["bertopic_top_topics"] = df.index.map(lambda i: topn_map.get(i, [-1]*topN))
    # expand top rank columns
    for i in range(topN):
        df[f"bertopic_topic_rank_{i+1}"] = df["bertopic_top_topics"].apply(lambda l: int(l[i]) if i < len(l) else -1)
    return df

In [7]:
# ----------------------------
# Example run (replace with your main flow)
# ----------------------------
df = load_data(DATA_PATH)
df = filter_by_docket(df, DOCKET_TO_USE)
df = df.dropna(subset=[TEXT_COL]).reset_index(drop=True)
docs = df[TEXT_COL].astype(str).tolist()

topic_model, chunks, origin_ids, embeddings, topics, probs = train_bertopic_on_chunks(
    docs,
    EMBEDDING_MODEL,
    chunk_size=CHUNK_SIZE,
    overlap=CHUNK_OVERLAP,
    umap_n_neighbors=UMAP_N_NEIGHBORS,
    umap_min_dist=UMAP_MIN_DIST,
    umap_n_components=UMAP_N_COMPONENTS,
    hdb_min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE,
    hdb_min_samples=HDBSCAN_MIN_SAMPLES,
    verbose=True
)

Loaded 12437 comments from C:\Users\linna\OneDrive\Documents\Python_Dev\topic-modeling\data\public_comments.json
Filtered to docket 'TTB-2025-0003': 189 comments
Created 407 chunks from 189 documents (ratio 2.15).


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

2025-09-08 14:19:08,503 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-08 14:19:40,528 - BERTopic - Dimensionality - Completed ✓
2025-09-08 14:19:40,534 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-08 14:19:41,123 - BERTopic - Cluster - Completed ✓
2025-09-08 14:19:41,141 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-08 14:19:41,623 - BERTopic - Representation - Completed ✓


BERTopic (chunks) training complete. Non-outlier topics: 62


In [8]:
# aggregate chunk-level topics back to original df rows
df_with_topics = aggregate_chunk_topics_to_docs(chunks, origin_ids, topics, probs, df, DOC_ID_COL, topN=3)

# build topic summary (you can reuse your build_topic_summary_bertopic function,
# but when chunking, representative docs will be chunk-level; prefer sampling from original docs)
topic_summary = build_topic_summary_bertopic(topic_model, df_with_topics, chunks, DOC_ID_COL, top_words=TOP_WORDS_PER_TOPIC, sample_docs=SAMPLE_DOCS_PER_TOPIC)

# save outputs
topic_summary.to_csv(TOPIC_SUMMARY_CSV, index=False)
df_with_topics.to_csv(OUTPUT_DF_CSV, index=False)
topic_model.save(str(MODEL_SAVE_FILE))
print("Saved outputs. Topics:", len(topic_model.get_topic_info()) - (1 if -1 in topic_model.get_topic_info()["Topic"].tolist() else 0))

2025-09-08 14:19:42,994 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Saved outputs. Topics: 62


In [9]:
# always an option to consolidate -- usually somewhere between 20-30 works well
# topic_model.reduce_topics(chunks, nr_topics=15)